# Lexicons

Lexicon definitions live in `lexicons` directory. tinychat's base NSID is:

`chat.tinychat`

Lexicons specs get converted to TS using

`deno task generate:lexicons`

**Note:** I've also included `com.atproto` lexicons - otherwise generated API
code for tinychat references types that are not available (for things like
record CRUD ops)

Random notes:

```
   chat.tinychat.actor

   chat.tinychat.server <-- chat servers I created 
      - tinychat dev channel
      - trumpet players

 
   chat.tinychat.channel
      - general @ tinychat dev channel
      - random @ tinychat dev channel

   chat.tinychat.message (each user's message are on his/her data repo)
      - DID identifies sender

   chat.tinychat.membership
      (use strong refs to reference server: https://github.com/bluesky-social/atproto/blob/main/lexicons/app/bsky/feed/like.json#L13)
      - server
```

## Using lexicons

Validate record

In [ ]:
import { assert } from "asserts";
import { validateRecord } from "tinychat/api/types/chat/tinychat/core/server.ts";
import { TID } from "@atproto/common";


Deno.test("validate records using lexicons", () => {
  // names must be a string + missing channels
  let r = validateRecord({
    name: 5,    
  });
  // @ts-ignore error is not defined
  assert(!r.success, r.error);

  // is required
  r = validateRecord({});
  // @ts-ignore error is not defined
  assert(!r.success, r.error);

  // all good
  r = validateRecord({
    name: "test",
    channels: [
      {
        name: "test",
        id: TID.nextStr(),
      },
    ],
  });
  // @ts-ignore error is not defined
  assert(r.success, r.error);
});

Let's test CRUD

In [ ]:
import { TinychatAgent } from "tinychat/agent.ts";
import { TID } from "@atproto/common";
import { assert, assertRejects } from "asserts";
import { validateRecord } from "tinychat/api/types/chat/tinychat/core/server.ts";

Deno.test("test CRUD", async () => {
  const ta = await TinychatAgent.create();
  const name = `test-${TID.nextStr()}`;

  const r = await ta.chat.tinychat.core.server.create(
    {
      repo: ta.agent.assertDid,
    },
    {
      name,
      channels: [
        {
          name: "test",
          id: TID.nextStr(),
        },
      ],
    }
  );

  const rkey = r.uri.split("/").pop() || "";

  const rec = await ta.chat.tinychat.core.server.get({
    repo: ta.agent.assertDid,
    rkey,
  });

  assert(rec);
  assert(rec.value.name === name);
  assert(rec.uri === r.uri);

  const { records } = await ta.chat.tinychat.core.server.list({
    repo: ta.agent.assertDid,
    limit: 10,
  });

  assert(records.length <= 10);
  assert(records.find((r) => r.uri === rec.uri));

  await ta.chat.tinychat.core.server.delete({
    repo: ta.agent.assertDid,
    rkey,
  });

  assert(
    !(
      await ta.chat.tinychat.core.server.list({
        repo: ta.agent.assertDid,
        limit: 10,
      })
    ).records.find((r) => r.uri === rec.uri),
  );
});

Deno.test("test validation", async () => {
  const ta = await TinychatAgent.create();

  await assertRejects(() =>
    ta.chat.tinychat.core.server.create(
      { repo: ta.agent.assertDid },
      // @ts-ignore yolo
      validateRecord({ name: 5 }).value,
    )
  );
});

Deno.test("test basic data model", async () => {
  const ta = await TinychatAgent.create();
  const repo = ta.agent.assertDid;
  const channelId = TID.nextStr();

  // create a server
  const { uri } = await ta.chat.tinychat.core.server.create(
    { repo },
    {
      name: "tinychat dev community",
      channels: [
          {
            name: "test",
            id: channelId,
          }
      ],
    }
  );

  // join it

  await ta.chat.tinychat.core.membership.create({ repo }, {
    server: uri,
    createdAt: new Date().toISOString(),
  });

  // add channels to the server
  
  // message the channel
  const message = await ta.chat.tinychat.core.message.create(
    { repo },
    {
      server: uri,
      channel: channelId,
      text: "hello world",
      createdAt: new Date().toISOString(),
    }
  );

  // reply to it
  await ta.chat.tinychat.core.message.create(
    { repo },
    {
      channel: channelId,
      text: "nice post",
      server: uri,
      createdAt: new Date().toISOString(),
      reply: {
        root: { uri: message.uri, cid: message.cid },
        parent: { uri: message.uri, cid: message.cid },
      },
    }
  );
});